In [ ]:
# | default_exp _components.encoder.json

In [ ]:
# | export


import json
from typing import *

from pydantic import BaseModel
from pydantic.main import ModelMetaclass

from fastkafka._components.logger import get_logger
from fastkafka._components.meta import export

In [ ]:
import json
import tempfile

from pydantic import Field

from fastkafka._components.logger import suppress_timestamps

In [ ]:
# | export

logger = get_logger(__name__)

In [ ]:
suppress_timestamps()
logger = get_logger(__name__, level=20)
logger.info("ok")

[INFO] __main__: ok


In [ ]:
# | export


def _to_json_utf8(o: Any) -> bytes:
    """Converts to JSON and then encodes with UTF-8"""
    if hasattr(o, "json"):
        return o.json().encode("utf-8")  # type: ignore
    else:
        return json.dumps(o).encode("utf-8")

In [ ]:
assert _to_json_utf8({"a": 1, "b": [2, 3]}) == b'{"a": 1, "b": [2, 3]}'


class A(BaseModel):
    name: str = Field()
    age: int


assert _to_json_utf8(A(name="Davor", age=12)) == b'{"name": "Davor", "age": 12}'

In [ ]:
class User(BaseModel):
    name: str
    favorite_number: Optional[int] = None
    favorite_color: Optional[str] = None

In [ ]:
# | export


@export("fastkafka.encoder")
def json_encoder(msg: BaseModel) -> bytes:
    """
    Encoder to encode pydantic instances to json string

    Args:
        msg: An instance of pydantic basemodel

    Returns:
        Json string in bytes which is encoded from pydantic basemodel
    """
    return _to_json_utf8(msg)

In [ ]:
msg = User(name="Kumaran", favorite_number=9, favorite_color="black")


actual = json_encoder(msg)
display(actual)

assert isinstance(actual, bytes)
assert actual == b'{"name": "Kumaran", "favorite_number": 9, "favorite_color": "black"}'

b'{"name": "Kumaran", "favorite_number": 9, "favorite_color": "black"}'

In [ ]:
# | export


@export("fastkafka.encoder")
def json_decoder(raw_msg: bytes, cls: ModelMetaclass) -> Any:
    """
    Decoder to decode json string in bytes to pydantic model instance

    Args:
        raw_msg: Bytes message received from Kafka topic
        cls: Pydantic class; This pydantic class will be used to construct instance of same class

    Returns:
        An instance of given pydantic class
    """
    msg_dict = json.loads(raw_msg.decode("utf-8"))

    return cls(**msg_dict)

In [ ]:
raw_msg = b'{"name": "123", "favorite_number": 0, "favorite_color": "111"}'


actual = json_decoder(raw_msg, cls=User)
display(actual)

assert isinstance(actual, User)
assert actual.name == "123"
assert actual.favorite_number == 0
assert actual.favorite_color == "111"

User(name='123', favorite_number=0, favorite_color='111')